# Classifying images with Caffe

We will classify some images with the pretrained GoogLeNet model that is available from [here](https://github.com/BVLC/caffe/tree/master/models/bvlc_googlenet). GoogLeNet is an convolutional network architecture that won the Imagenet competition in 2014. For more details on the architecture, see [this paper](http://arxiv.org/abs/1409.4842).

![img](./images/googlenet_diagram.png)

We assume that we're running from EC2 AMI, otherwise you need to download the googlenet model and set ```caffe_root``` and ```model_root``` variables below.

What we need for a pretrained model, is these files:
* deploy.prototxt - the model definition file
* bvlc_googlenet.caffemodel - a binary file with the learned model parameters


In [1]:
!cat ~/caffe/models/bvlc_googlenet/deploy.prototxt

name: "GoogleNet"
input: "data"
input_shape {
  dim: 10
  dim: 3
  dim: 224
  dim: 224
}
layer {
  name: "conv1/7x7_s2"
  type: "Convolution"
  bottom: "data"
  top: "conv1/7x7_s2"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 64
    pad: 3
    kernel_size: 7
    stride: 2
    weight_filler {
      type: "xavier"
      std: 0.1
    }
    bias_filler {
      type: "constant"
      value: 0.2
    }
  }
}
layer {
  name: "conv1/relu_7x7"
  type: "ReLU"
  bottom: "conv1/7x7_s2"
  top: "conv1/7x7_s2"
}
layer {
  name: "pool1/3x3_s2"
  type: "Pooling"
  bottom: "conv1/7x7_s2"
  top: "pool1/3x3_s2"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "pool1/norm1"
  type: "LRN"
  bottom: "pool1/3x3_s2"
  top: "pool1/norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "conv2/3x3_reduce"
  type: "Convolution"
  bottom: "pool1/norm1"
  

In [7]:
import numpy as np

# Make sure that caffe is on the python path:
caffe_root = '/home/ubuntu/caffe/' # this file is expected to be in {caffe_root}/examples
model_root = '/home/ubuntu/caffe/models/bvlc_googlenet/'
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

In [2]:
caffe.set_mode_gpu()
net = caffe.Net(model_root + 'deploy.prototxt', 
                model_root + 'bvlc_googlenet.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB (opencv)

We want to classify this image of a norwegian elkhound:

![](./images/norwegian_elkhound_small.jpg)

Feed in the image (with some preprocessing) and classify with a forward pass.

In [4]:
im = caffe.io.load_image('./images/norwegian_elkhound_small.jpg')
#im = caffe.io.load_image('./images/ant_small.jpg')
#im = caffe.io.load_image('./images/strawberry_small.jpg')
im = transformer.preprocess('data', im)
net.blobs['data'].data[...] = im
out = net.forward()
out['prob']
print("Predicted class is #{}.".format(out['prob'].argmax()))

Predicted class is #174.


In [5]:
# load labels
imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

# sort top k predictions from softmax output
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
#print labels[top_k]
for k in top_k:
    label_string = labels[k][10:].split(",")[0]
    print "%s : %f.3" % (label_string, net.blobs['prob'].data[0].flatten()[k])

Norwegian elkhound : 0.999874.3
malamute : 0.000060.3
keeshond : 0.000015.3
German shepherd : 0.000014.3
malinois : 0.000013.3


Another image, a magpie:

![](./images/magpie_small.jpg)

In [6]:
im = caffe.io.load_image('./images/magpie_small.jpg')
im = transformer.preprocess('data', im)
net.blobs['data'].data[...] = im
out = net.forward()
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
#print labels[top_k]
for k in top_k:
    label_string = labels[k][10:].split(",")[0]
    print "%s : %f.3" % (label_string, net.blobs['prob'].data[0].flatten()[k])

magpie : 0.999774.3
water ouzel : 0.000080.3
oystercatcher : 0.000045.3
jay : 0.000022.3
toucan : 0.000020.3


A castle:

![](./images/castle_small.jpg)

In [8]:
im = caffe.io.load_image('./images/castle_small.jpg')
im = transformer.preprocess('data', im)
net.blobs['data'].data[...] = im
out = net.forward()
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
#print labels[top_k]
for k in top_k:
    label_string = labels[k][10:].split(",")[0]
    print "%s : %f.3" % (label_string, net.blobs['prob'].data[0].flatten()[k])

castle : 0.964401.3
palace : 0.024251.3
monastery : 0.010008.3
church : 0.000948.3
bell cote : 0.000097.3


In [9]:
%%time
# CPU mode
caffe.set_mode_cpu()
net.forward() # call once for allocation

CPU times: user 6.51 s, sys: 78.9 ms, total: 6.58 s
Wall time: 6.58 s


In [11]:
%%time
# GPU mode
caffe.set_mode_gpu()
net.forward()  # call once for allocation

CPU times: user 51.8 ms, sys: 24.1 ms, total: 75.8 ms
Wall time: 75.5 ms


In [6]:
print labels[0:100]

['n01440764 tench, Tinca tinca' 'n01443537 goldfish, Carassius auratus'
 'n01484850 great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias'
 'n01491361 tiger shark, Galeocerdo cuvieri'
 'n01494475 hammerhead, hammerhead shark'
 'n01496331 electric ray, crampfish, numbfish, torpedo'
 'n01498041 stingray' 'n01514668 cock' 'n01514859 hen'
 'n01518878 ostrich, Struthio camelus'
 'n01530575 brambling, Fringilla montifringilla'
 'n01531178 goldfinch, Carduelis carduelis'
 'n01532829 house finch, linnet, Carpodacus mexicanus'
 'n01534433 junco, snowbird'
 'n01537544 indigo bunting, indigo finch, indigo bird, Passerina cyanea'
 'n01558993 robin, American robin, Turdus migratorius' 'n01560419 bulbul'
 'n01580077 jay' 'n01582220 magpie' 'n01592084 chickadee'
 'n01601694 water ouzel, dipper' 'n01608432 kite'
 'n01614925 bald eagle, American eagle, Haliaeetus leucocephalus'
 'n01616318 vulture'
 'n01622779 great grey owl, great gray owl, Strix nebulosa'
 'n01629819 Eur